# 22 春日部防衛隊（かすかべ防衛隊）

- GAOZの組みを全く変えていない
  -> 多数決方式でせめる  
- ラプラスノイズをかけてるのかな？
- election with ignorance方式で攻めよう

### 担当者と日付
2024/10/2 渡邉健

In [52]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [53]:
# fmt: off
ALL_HEADER = ['Name', 'Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '247', '260', '653', '673', '810', '885',  # noqa: E501
                      '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017',  # noqa: E501
                      '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253',  # noqa: E501
                      '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489',  # noqa: E501
                      '3877', '3889'] # noqa

B_HEADERS_LIST = [
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '1920', '2017', '2087'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

B_USER_ATTRIBUTE_HEADERS = ['Gender', 'Age', 'Occupation', 'ZIP-code']

B_REVIEW_HEADERS_LIST = [
  ['260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['247', '1920', '2017', '2087'],  # noqa: E501
  ['260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

MOVIE_IDS = ['2', '56', '247', '260', '653', '673', '810', '885', '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253', '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489', '3877', '3889']
# fmt: on


### C_0 ~ C_9までの読み込み

In [54]:
from os import listdir

ATTACK_TARGET = "22"

c_data_list = []

files = listdir("../data")

for i in range(10):
    c_data = pd.read_csv(f"../data/C{ATTACK_TARGET}_{i}.csv")
    file = None
    for f in files:
        if f.startswith(f"C{ATTACK_TARGET}_{i}"):
            file = f
            break
    if file is None:
        print(f)
        raise("File not found")
    c_data_list.append(c_data)

c_data_list[0]

,Gender,Age,Occupation,ZIP-code,260,653,1525,2105,2193,2253,2628,2872,3438,3439,3440,3877,3889
0,F,25.0,0.0,554.0,0.0,4.0,1.0,4.0,5.0,1.0,5.0,0.0,3.0,3.0,0.0,0.0,0.0
1,M,25.0,11.0,904.0,2.0,2.0,0.0,4.0,1.0,0.0,0.0,0.0,3.0,4.0,0.0,3.0,0.0
2,F,1.0,13.0,525.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,4.0,4.0,3.0,0.0,1.0,0.0
3,F,45.0,8.0,261.0,1.0,0.0,1.0,4.0,5.0,0.0,3.0,4.0,0.0,2.0,5.0,0.0,1.0
4,F,25.0,5.0,273.0,5.0,4.0,4.0,2.0,0.0,1.0,0.0,0.0,4.0,2.0,3.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10002,NaN,NaN,NaN,1.0,1713.0,1116.0,1196.0,1596.0,879.0,1321.0,1383.0,757.0,1742.0,1105.0,882.0,995.0,1857.0
10003,NaN,NaN,NaN,2.0,1715.0,1509.0,1410.0,1630.0,1265.0,1067.0,901.0,1383.0,1779.0,1457.0,1453.0,766.0,1196.0
10004,NaN,NaN,NaN,3.0,1638.0,1008.0,1559.0,1741.0,1693.0,1133.0,1491.0,801.0,1542.0,1053.0,890.0,1088.0,748.0
10005,NaN,NaN,NaN,4.0,1565.0,1235.0,1233.0,1784.0,1402.0,1321.0,2632.0,985.0,1518.0,1005.0,798.0,1100.0,542.0


### Ba, Bbの読み込み

In [55]:
Ba = pd.read_csv(f"../data/B{ATTACK_TARGET}a.csv", names=ALL_HEADER)
Ba = Ba.drop(0)
Ba.head()

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
1,Efren Mityukov,M,1,16,171,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adella Edser,F,18,8,658,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Mavis Dever,F,18,17,985,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Merell Burless,M,18,17,180,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Esma Dillow,F,25,0,752,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
Bb = pd.read_csv(f"../data/B{ATTACK_TARGET}b.csv")
Bb.head()

,2,56,247,260,653,673,810,885,1009,1073,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,1,2,3,3,2,0,0,5,3,*,...,0,0,4,1,0,3,1,1,0,0
1,2,0,0,3,0,0,0,0,0,2,...,0,0,1,0,0,0,4,0,0,0
2,0,3,0,5,1,0,0,5,3,3,...,2,1,2,0,0,5,1,0,5,1
3,2,2,1,4,4,0,0,0,*,5,...,4,0,2,0,1,0,1,3,2,2
4,4,0,3,1,3,5,2,4,2,2,...,2,1,5,0,0,5,0,1,3,0


### BaのN番目の行と完全一致する行をC0~C9から取得

In [57]:
# Baの2行目を選択
row_index = 4

gender = Ba.iloc[row_index]["Gender"]
age = Ba.iloc[row_index]["Age"]
occupation = Ba.iloc[row_index]["Occupation"]
zipcode = Ba.iloc[row_index]["ZIP-code"]

print("検索条件")
print(gender, age, occupation, zipcode)

filtered_C_list = []

# 完全一致を見つける
for c in c_data_list:
    filtered_C_0 = c[(c['Gender'] == gender) & 
                      (c['Age'] == int(age)) & 
                      (c['Occupation'] == int(occupation)) & 
                      (c['ZIP-code'] == int(zipcode))]
    filtered_C_list.append(filtered_C_0)

print("完全一致した件数 (10であることが望まれる)")
print(len(filtered_C_list))

検索条件
F 25 0 752
完全一致した件数 (10であることが望まれる)
10


### 各カラムごとに多数決の候補を表示する

In [58]:
merged_rows = []

for i, filtered_df in enumerate(filtered_C_list):
    if not filtered_df.empty:
        # 各フィルタリングされたデータフレームから1行だけ取得
        row = filtered_df.iloc[0]
        
        # ALL_HEADERに基づいて新しい行を作成
        new_row = pd.Series(index=ALL_HEADER)
        
        # データを転送
        for col in ALL_HEADER:
            if col in filtered_df.columns:
                new_row[col] = row[col]
            else:
                new_row[col] = np.nan
        
        # 新しい行をリストに追加
        merged_rows.append(new_row)

# リストからデータフレームを作成
merged_df = pd.DataFrame(merged_rows, columns=ALL_HEADER)
merged_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,F,25.0,0.0,752.0,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,3.0,3.0,0.0,NaN,NaN,NaN,0.0,0.0
1,NaN,F,25.0,0.0,752.0,5.0,1.0,NaN,1.0,3.0,...,1.0,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0
2,NaN,F,25.0,0.0,752.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,F,25.0,0.0,752.0,2.0,0.0,NaN,NaN,NaN,...,NaN,NaN,3.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,NaN,F,25.0,0.0,752.0,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
5,NaN,F,25.0,0.0,752.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,F,25.0,0.0,752.0,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
7,NaN,F,25.0,0.0,752.0,NaN,NaN,NaN,0.0,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,F,25.0,0.0,752.0,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,F,25.0,0.0,752.0,NaN,NaN,NaN,0.0,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### ざっと見て、多数決の候補にばらつきはなさそうと判断し、各行に対してランダムに一つ値を取得できるようにする

In [59]:
def select_random_value(column):
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if valid_values:
        # 有効な値がある場合、ランダムに1つ選択
        return np.random.choice(valid_values)
    else:
        # 有効な値がない場合はNaNを返す
        return np.nan

# 各カラムからランダムに値を選択
random_values = {}
for column in merged_df.columns:
    random_values[column] = select_random_value(merged_df[column])

# 結果を新しいデータフレームとして作成
result_df = pd.DataFrame([random_values])
result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,F,25.0,0.0,752.0,5.0,1.0,3.0,0.0,1.0,...,5.0,0.0,3.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0


### 以上の処理をBaの各行（0~49行）に対して実行する

In [60]:
import pandas as pd
import numpy as np

final_df = pd.DataFrame(columns=ALL_HEADER)

for row_index in range(50):
    # Baデータフレームから現在の行の情報を取得
    gender = Ba.iloc[row_index]["Gender"]
    age = Ba.iloc[row_index]["Age"]
    occupation = Ba.iloc[row_index]["Occupation"]
    zipcode = Ba.iloc[row_index]["ZIP-code"]

    filtered_C_list = []

    # 各cデータフレームに対して、完全一致するレコードをフィルタリング
    for c in c_data_list:
        filtered_C_0 = c[(c['Gender'] == gender) & 
                        (c['Age'] == int(age)) & 
                        (c['Occupation'] == int(occupation)) & 
                        (c['ZIP-code'] == int(zipcode))]
        filtered_C_list.append(filtered_C_0)

    # 多数決の候補となる行のリストを作成
    candidate_rows = []
    for i, filtered_df in enumerate(filtered_C_list):
        if not filtered_df.empty:
            # 各フィルタリングされたデータフレームから1行だけ取得
            row = filtered_df.iloc[0]
            # ALL_HEADERに基づいて新しい行を作成し、データを転送
            candidate_row = pd.Series(index=ALL_HEADER)
            for col in ALL_HEADER:
                candidate_row[col] = row[col] if col in filtered_df.columns else np.nan
            # 新しい行を候補リストに追加
            candidate_rows.append(candidate_row)

    # 候補行のリストからデータフレームを作成
    candidate_rows_df = pd.DataFrame(candidate_rows, columns=ALL_HEADER)

    # 各列に対して、ランダムに値を選択（多数決の実装）
    elected_value = {}
    for column in ALL_HEADER:
        elected_value[column] = select_random_value(candidate_rows_df[column])

    # 選択された値から新しい行を作成
    elected_value_row_df = pd.DataFrame([elected_value])

    # 選択された行をfinal_dfの最後に追加
    final_df = pd.concat([final_df, elected_value_row_df], ignore_index=True)

final_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,NaN,M,1.0,16.0,171.0,5.0,4.0,1.0,5.0,1.0,...,5.0,5.0,5.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0
1,NaN,F,18.0,8.0,658.0,4.0,4.0,5.0,5.0,2.0,...,0.0,2.0,2.0,3.0,2.0,2.0,5.0,1.0,1.0,0.0
2,NaN,F,18.0,17.0,985.0,1.0,0.0,3.0,0.0,3.0,...,0.0,0.0,5.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0
3,NaN,M,18.0,17.0,180.0,1.0,0.0,5.0,2.0,3.0,...,5.0,1.0,2.0,0.0,2.0,0.0,4.0,1.0,1.0,2.0
4,NaN,F,25.0,0.0,752.0,2.0,0.0,1.0,0.0,3.0,...,5.0,0.0,3.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0
5,NaN,F,56.0,8.0,85.0,0.0,0.0,4.0,1.0,0.0,...,0.0,0.0,1.0,2.0,2.0,3.0,4.0,0.0,4.0,4.0
6,NaN,M,25.0,1.0,100.0,0.0,3.0,2.0,1.0,0.0,...,2.0,5.0,0.0,3.0,3.0,4.0,3.0,0.0,0.0,5.0
7,NaN,F,35.0,5.0,590.0,2.0,3.0,5.0,3.0,2.0,...,5.0,0.0,4.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0
8,NaN,M,18.0,8.0,840.0,0.0,0.0,1.0,4.0,0.0,...,2.0,1.0,1.0,0.0,4.0,5.0,5.0,4.0,0.0,2.0
9,NaN,F,56.0,1.0,436.0,3.0,0.0,4.0,0.0,3.0,...,0.0,1.0,1.0,3.0,0.0,0.0,4.0,1.0,3.0,0.0


### 最終的に得た映画のレビューの候補とBbを比較し、Bbの何番目のindexが一番近くてハミング距離にどれくらいの違いがあるかを取得する

In [61]:

def hamming_distance(a, b):
    a_int = a.fillna(0).astype(int)
    b_int = b.replace('*', None).fillna(0).astype(int)
    # ハミング距離の計算
    return np.sum(a_int != b_int)

def find_closest_row(row, Bb):
    distances = Bb.apply(lambda bb_row: hamming_distance(row[MOVIE_IDS], bb_row[MOVIE_IDS]), axis=1)
    min_distance = distances.min()
    closest_index = distances.idxmin()
    return pd.Series({'Min_Hamming_Distance': min_distance, 'Closest_Bb_Index': closest_index})

# 結果を格納するデータフレーム
result_df = pd.DataFrame(index=final_df.index, columns=['Min_Hamming_Distance', 'Closest_Bb_Index'])

# 各行に対してハミング距離を計算し、最も近い行を見つける
for idx, row in final_df[MOVIE_IDS].iterrows():
    result = find_closest_row(row, Bb[MOVIE_IDS])
    result_df.loc[idx] = result

# 元のfinal_dfにハミング距離の結果を結合
final_result_df = pd.concat([final_df, result_df], axis=1)
final_result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,Min_Hamming_Distance,Closest_Bb_Index
0,NaN,M,1.0,16.0,171.0,5.0,4.0,1.0,5.0,1.0,...,5.0,0.0,0.0,1.0,0.0,4.0,0.0,0.0,30,9
1,NaN,F,18.0,8.0,658.0,4.0,4.0,5.0,5.0,2.0,...,2.0,3.0,2.0,2.0,5.0,1.0,1.0,0.0,30,5
2,NaN,F,18.0,17.0,985.0,1.0,0.0,3.0,0.0,3.0,...,5.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,21,30
3,NaN,M,18.0,17.0,180.0,1.0,0.0,5.0,2.0,3.0,...,2.0,0.0,2.0,0.0,4.0,1.0,1.0,2.0,29,14
4,NaN,F,25.0,0.0,752.0,2.0,0.0,1.0,0.0,3.0,...,3.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,21,33
5,NaN,F,56.0,8.0,85.0,0.0,0.0,4.0,1.0,0.0,...,1.0,2.0,2.0,3.0,4.0,0.0,4.0,4.0,30,23
6,NaN,M,25.0,1.0,100.0,0.0,3.0,2.0,1.0,0.0,...,0.0,3.0,3.0,4.0,3.0,0.0,0.0,5.0,29,21
7,NaN,F,35.0,5.0,590.0,2.0,3.0,5.0,3.0,2.0,...,4.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,29,0
8,NaN,M,18.0,8.0,840.0,0.0,0.0,1.0,4.0,0.0,...,1.0,0.0,4.0,5.0,5.0,4.0,0.0,2.0,29,20
9,NaN,F,56.0,1.0,436.0,3.0,0.0,4.0,0.0,3.0,...,1.0,3.0,0.0,0.0,4.0,1.0,3.0,0.0,26,30
